In [1]:
# Download the corresponding PyTorch Geometric module
%%capture
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.1+cu113
"""
%env TORCH=2.1.0+cu118
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
import torch.nn.functional as F

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
# # Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


# Check if CUDA is available and use it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the data to the device (GPU if available)
data = data.to(device)

Processing...
Done!


In [4]:
import pdb
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv, global_mean_pool, global_max_pool
import random
from torch_geometric.utils import to_dense_adj
from sklearn.metrics import f1_score
import numpy as np

class GINMLP(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GINMLP, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, output_dim)
        self.relu = torch.nn.ReLU()
        # self.linear2 = torch.nn.Linear(output_dim, output_dim)

    def forward(self, x):
        x = self.linear1(x)
        # x = self.relu(x)
        # x = self.linear2(x)
        return x
    def reset_parameters(self):
        """Resets the parameters of the linear layers within the MLP."""
        self.linear1.reset_parameters()
        # self.linear2.reset_parameters()

class GIN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GIN, self).__init__()
        self.conv1 = GINConv(GINMLP(input_dim, hidden_dim))
        self.conv2 = GINConv(GINMLP(hidden_dim, hidden_dim))
        self.lin = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x) # Apply activation after GINConv
        x = self.conv2(x, edge_index)
        x = F.relu(x) # Apply activation after GINConv
        x = self.lin(x)
        return x

    def get_ebd(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x) # Apply activation after GINConv
        x = self.conv2(x, edge_index)
        x = F.relu(x) # Apply activation after GINConv
        return x

In [5]:
# Training the model
def train(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        # Get the node features (data.x) and edge indices (data.edge_index)
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluate the model
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = correct / data.test_mask.sum()
        return acc.item()

def reset_weights(m):
    if isinstance(m, (torch.nn.Linear)):
        m.reset_parameters()

In [6]:
# Model hyperparameters
in_channels = dataset.num_node_features  # 1433 (Cora input feature size)
hidden_channels = 64
out_channels = dataset.num_classes  # 7 (Cora has 7 classes)
# Initialize the model
model = GIN(in_channels, hidden_channels, out_channels).to(device)
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [7]:
# Train the model
model.apply(reset_weights)
train(model, data, optimizer, criterion, epochs=50)

# Evaluate the model
acc = evaluate(model, data)
print(f'Test accuracy: {acc:.4f}')

Epoch 1/50, Loss: 1.9508
Epoch 11/50, Loss: 0.2746
Epoch 21/50, Loss: 0.0101
Epoch 31/50, Loss: 0.0006
Epoch 41/50, Loss: 0.0003
Test accuracy: 0.7440


In [8]:
from torch_geometric.utils import to_dense_adj
# Node features (X)
x = data.x  # Shape: [num_nodes, num_features]
# Adjacency matrix (A)
A = to_dense_adj(data.edge_index)[0]  # Shape: [num_nodes, num_nodes]
K = (data.x) @ (data.x).T
m = 1000

In [9]:
from Perturbe_Algs import Attacks
from copy import deepcopy
attack_instance = Attacks(A,K,m, alpha = 50, filter = 'adj', max_iter=250,).to(device)


In [ ]:
pgdavg_dataset = deepcopy(data)
pgdwst_dataset = deepcopy(data)

A_pgd_avg = attack_instance.Prob_PGD().clone().detach()
A_pgd_wst = attack_instance.Wst_PGD().clone().detach()

pgdavg_dataset.edge_index = torch.tensor(A_pgd_avg).nonzero(as_tuple=False).t().contiguous()
pgdwst_dataset.edge_index = torch.tensor(A_pgd_wst).nonzero(as_tuple=False).t().contiguous()

# Save the dataset object
# Save the adjacency matrix (A)
torch.save(A_pgd_avg, 'avg_adj.pt')
torch.save(A_pgd_wst, 'wst_adj.pt')

# Save the modified datasets
torch.save(pgdavg_dataset, 'cora_pgdavg_data.pt')
torch.save(pgdwst_dataset, 'cora_pgdwst_data.pt')

print("Dataset and adjacency matrix saved.")

/content/Perturbe_Algs.py:248: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at /pytorch/aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:690.)
  S_iter = S_iter + (a/torch.linalg.norm(S_iter.grad, ord=2)) * S_iter.grad


Dataset and adjacency matrix saved.


<ipython-input-40-97ca172e3aca>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pgdavg_dataset.edge_index = torch.tensor(A_pgd_avg).nonzero(as_tuple=False).t().contiguous()
<ipython-input-40-97ca172e3aca>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pgdwst_dataset.edge_index = torch.tensor(A_pgd_wst).nonzero(as_tuple=False).t().contiguous()


In [10]:
# Load the attacked graph adjacency matrix from file 'cora_adj'
# pgdavg_dataset = torch.load('cora_pgdavg_data.pt', weights_only=False)
# pgdwst_dataset = torch.load('cora_pgdwst_data.pt', weights_only=False)

In [11]:
num_rep = 10

list_ebd_random = []
list_ebd_pgdavg = []
list_ebd_pgdwst = []
list_acc = []
list_acc_random = []
list_acc_pgdavg = []
list_acc_pgdwst = []

for i in range(num_rep):

  A_random = attack_instance.randomAttack().clone().detach()
  random_dataset = deepcopy(data)
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()

  model.apply(reset_weights)
  train(model, data, optimizer, criterion, epochs=50)
  test_acc = evaluate(model, data)
  acc_random = evaluate(model, random_dataset)
  acc_pgdavg = evaluate(model, pgdavg_dataset)
  acc_pgdwst = evaluate(model, pgdwst_dataset)

  list_acc.append(test_acc)
  list_acc_random.append(acc_random)
  list_acc_pgdavg.append(acc_pgdavg)
  list_acc_pgdwst.append(acc_pgdwst)


  ebd = model.get_ebd(data.x, data.edge_index)
  ebd_random = model.get_ebd(random_dataset.x, random_dataset.edge_index)
  ebd_pgdavg = model.get_ebd(pgdavg_dataset.x, pgdavg_dataset.edge_index)
  ebd_pgdwst = model.get_ebd(pgdwst_dataset.x, pgdwst_dataset.edge_index)


  p_ebd_random = torch.norm(ebd - ebd_random, p='fro')
  p_ebd_pgdavg = torch.norm(ebd - ebd_pgdavg, p='fro')
  p_ebd_pgdwst = torch.norm(ebd - ebd_pgdwst, p='fro')

  list_ebd_random.append(p_ebd_random.item())
  list_ebd_pgdavg.append(p_ebd_pgdavg.item())
  list_ebd_pgdwst.append(p_ebd_pgdwst.item())

  print(f"-----Iteration {i:d}-----")
  print(f"Test Acc: {test_acc:.4f}")
  print(f"PGD-AVG Test Acc: {acc_pgdavg:.4f}")
  print(f"PGD-WST Test Acc: {acc_pgdwst:.4f}")
  print(f"Random Test Acc: {acc_random:.4f}")



<ipython-input-11-2397148621>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()


Epoch 1/50, Loss: 1.9502
Epoch 11/50, Loss: 0.0904
Epoch 21/50, Loss: 0.0013
Epoch 31/50, Loss: 0.0020
Epoch 41/50, Loss: 0.0075
-----Iteration 0-----
Test Acc: 0.7220
PGD-AVG Test Acc: 0.5170
PGD-WST Test Acc: 0.7010
Random Test Acc: 0.6880
Epoch 1/50, Loss: 1.9348
Epoch 11/50, Loss: 0.4660
Epoch 21/50, Loss: 0.0128
Epoch 31/50, Loss: 0.0047
Epoch 41/50, Loss: 0.0010
-----Iteration 1-----
Test Acc: 0.7630
PGD-AVG Test Acc: 0.6400
PGD-WST Test Acc: 0.7550
Random Test Acc: 0.7330
Epoch 1/50, Loss: 2.0614
Epoch 11/50, Loss: 0.5282
Epoch 21/50, Loss: 0.0703
Epoch 31/50, Loss: 0.0092
Epoch 41/50, Loss: 0.0041
-----Iteration 2-----
Test Acc: 0.7510
PGD-AVG Test Acc: 0.6020
PGD-WST Test Acc: 0.7450
Random Test Acc: 0.7080
Epoch 1/50, Loss: 2.0349
Epoch 11/50, Loss: 0.0856
Epoch 21/50, Loss: 0.4447
Epoch 31/50, Loss: 0.0212
Epoch 41/50, Loss: 0.0079
-----Iteration 3-----
Test Acc: 0.7620
PGD-AVG Test Acc: 0.5340
PGD-WST Test Acc: 0.7530
Random Test Acc: 0.7230
Epoch 1/50, Loss: 1.9612
Epoch 1

In [15]:

print(f"Test: {np.mean(list_acc)*100:.2f}, std:{np.std(list_acc)*100:.2f}")
print(f"Mean of random: {np.mean(list_acc_random)*100:.2f}, std:{np.std(list_acc_random)*100:.2f}")
print(f"Mean of Wst_PGD: {np.mean(list_acc_pgdwst)*100:.2f}, std: {np.std(list_acc_pgdwst)*100:.2f}")
print(f"Mean of Prob_PGD:{np.mean(list_acc_pgdavg)*100:.2f}, std: {np.std(list_acc_pgdavg)*100:.2f}")


Test: 74.92, std:1.86
Mean of random: 71.75, std:1.85
Mean of Wst_PGD: 73.76, std: 2.07
Mean of Prob_PGD:56.67, std: 4.57


In [16]:
print(f"Mean of random embedding norm: {np.mean(list_ebd_random):.2f}, std:{np.std(list_ebd_random):.2f}")
print(f"Mean of Wst_PGD embedding norm: {np.mean(list_ebd_pgdwst):.2f}, std: {np.std(list_ebd_pgdwst):.2f}")
print(f"Mean of Prob_PGD embedding norm:{np.mean(list_ebd_pgdavg):.2f}, std: {np.std(list_ebd_pgdavg):.2f}")

Mean of random embedding norm: 974.49, std:496.47
Mean of Wst_PGD embedding norm: 16901.93, std: 12495.32
Mean of Prob_PGD embedding norm:43565.20, std: 33035.03


In [17]:
print(f"Test: {np.mean(list_acc)*100:.2f} \pm {np.std(list_acc)*100:.2f}")
print(f"Mean of random: {np.mean(list_acc_random)*100:.2f} \pm {np.std(list_acc_random)*100:.2f}")
print(f"Mean of Wst_PGD: {np.mean(list_acc_pgdwst)*100:.2f} \pm {np.std(list_acc_pgdwst)*100:.2f}")
print(f"Mean of Prob_PGD:{np.mean(list_acc_pgdavg)*100:.2f} \pm {np.std(list_acc_pgdavg)*100:.2f}")
print(f"Mean of random embedding norm: {np.mean(list_ebd_random):.2f} \pm {np.std(list_ebd_random):.2f}")
print(f"Mean of Wst_PGD embedding norm: {np.mean(list_ebd_pgdwst):.2f} \pm {np.std(list_ebd_pgdwst):.2f}")
print(f"Mean of Prob_PGD embedding norm:{np.mean(list_ebd_pgdavg):.2f} \pm {np.std(list_ebd_pgdavg):.2f}")

Test: 74.92 \pm 1.86
Mean of random: 71.75 \pm 1.85
Mean of Wst_PGD: 73.76 \pm 2.07
Mean of Prob_PGD:56.67 \pm 4.57
Mean of random embedding norm: 974.49 \pm 496.47
Mean of Wst_PGD embedding norm: 16901.93 \pm 12495.32
Mean of Prob_PGD embedding norm:43565.20 \pm 33035.03
